# ref: [Day 18】Embeddings 的家 - 向量資料庫](https://ithelp.ithome.com.tw/articles/10345832)
`pip install qdrant-client`

## Qdrant原生方法

In [2]:
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import VectorParams, Distance


embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# 連線到資料庫
client = QdrantClient(url="http://localhost:6333")

# 如果 collections 不存在可以就建立一個 collections
if client.collection_exists(collection_name="test1"):
	pass
else:
	client.create_collection(
		collection_name="test1",
		# The TWCC embedding model has 1536 dimensions
		vectors_config={
			"text": VectorParams(
				# size=1536,
				size=3072,
				distance=Distance.COSINE,
			),
		},
	)

# 欲匯入資料庫的資料
documents = [
	"Python 是一種高階編程語言，近年來被拿來做許多機器學習和深度學習的開發與模型訓練。",
	"Java 是一種面向對象的編程語言，廣泛應用於企業級應用和 Android 開發。",
	"JavaScript 是網頁開發的核心語言，允許動態操作 DOM 以實現交互效果。",
	"Go 是由 Google 開發的開源編程語言，因其高併發性能在雲端運算中大放異彩。",
	"C# 是由微軟開發的語言，主要用於 Windows 應用開發及遊戲開發，特別是在 Unity 引擎中。",
	"PHP 是一種伺服器端腳本語言，廣泛應用於網頁開發，尤其是動態內容生成。",
	# "貓貓可愛"
]

# 將資料全部匯入資料庫
for count, doc in enumerate(documents):
	client.upsert(
		collection_name="test1",
		points=[
			models.PointStruct(
				id=count+1,
				vector={"text": embeddings.embed_query(doc)},
				payload={"text": doc},
			)
		],
	)

## LangChain

In [2]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import VectorParams, Distance
from langchain_core.documents import Document
from uuid import uuid4


embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# 欲匯入資料庫的資料
documents = [
	Document(page_content="Python 是一種高階編程語言，近年來被拿來做許多機器學習和深度學習的開發與模型訓練。", metadata={"code":"Python"}),
	Document(page_content="Java 是一種面向對象的編程語言，廣泛應用於企業級應用和 Android 開發。", metadata={"code":"Java"}),
	Document(page_content="JavaScript 是網頁開發的核心語言，允許動態操作 DOM 以實現交互效果。", metadata={"code":"js"}),
	Document(page_content="Go 是由 Google 開發的開源編程語言，因其高併發性能在雲端運算中大放異彩。", metadata={"code":"Go"}),
	Document(page_content="C# 是由微軟開發的語言，主要用於 Windows 應用開發及遊戲開發，特別是在 Unity 引擎中。", metadata={"code":"C#"}),
	Document(page_content="PHP 是一種伺服器端腳本語言，廣泛應用於網頁開發，尤其是動態內容生成。", metadata={"code":"PHP"}),
]

## 方法一
# 連線到資料庫
client = QdrantClient(url="http://localhost:6333")

# 如果 collections 不存在可以就建立一個 collections
if client.collection_exists(collection_name="ithome2024_method_1"):
	pass
else:
	client.create_collection(
		collection_name="ithome2024_method_1",
		# The TWCC embedding model has 1536 dimensions
		vectors_config={
			"text": VectorParams(
				size=3072,
				distance=Distance.COSINE,
			),
		},
	)

# 建立 vector stores
vector_store = QdrantVectorStore(
	client=client,
	collection_name="ithome2024_method_1",
	embedding=embeddings,
	vector_name="text",
)

# 將資料轉向量匯入資料庫，Primary Key 給 uuid
ids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=ids)


## 方法二
qdrant = QdrantVectorStore.from_documents(
	documents,
	embeddings,
	url="localhost",
	collection_name="ithome2024_method_2",
)

* 可以看到使用 LangChain 會程式碼比較簡短和方便，但是這個方式就是設定 metadata，而不是 payload，但本質上意思是一樣的，都是存放需要的其餘資料。
* 這邊提供兩種方式，一種是還是需要透過 qdrant-client 去連線資料庫，而第二種方式指定本地端就可以自動上傳指定的地方，是最精簡的方式。
## 向量資料庫 Retriever 實戰

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# 可以直接連線建立好的 collections
vector_store = QdrantVectorStore.from_existing_collection(
	url="http://localhost:6333",
	collection_name="ithome2024_method_1",
	embedding=embeddings,
	vector_name="text",
)

# Cosine Similarity 的 Retriever
cos_retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 2})
print("這邊是 Cosine Similarity：")
result = cos_retriever.invoke('我想成為前端工程師！')
print(result[0].page_content, result[1].page_content)
result = vector_store.similarity_search(query='我想成為前端工程師！', k=2)
print(result[0].page_content, result[1].page_content)

# MMR 的 Retriever
mmr_retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={'k': 2, 'lambda_mult': 0.25})
print("\n這邊是 MMR：")
result = mmr_retriever.invoke('我想成為前端工程師！')
print(result[0].page_content, result[1].page_content)
result = vector_store.max_marginal_relevance_search(query='我想成為前端工程師！', k=2, lambda_mult=0.25)
print(result[0].page_content, result[1].page_content)

這邊是 Cosine Similarity：
JavaScript 是網頁開發的核心語言，允許動態操作 DOM 以實現交互效果。 Java 是一種面向對象的編程語言，廣泛應用於企業級應用和 Android 開發。
JavaScript 是網頁開發的核心語言，允許動態操作 DOM 以實現交互效果。 Java 是一種面向對象的編程語言，廣泛應用於企業級應用和 Android 開發。

這邊是 MMR：
JavaScript 是網頁開發的核心語言，允許動態操作 DOM 以實現交互效果。 Python 是一種高階編程語言，近年來被拿來做許多機器學習和深度學習的開發與模型訓練。
JavaScript 是網頁開發的核心語言，允許動態操作 DOM 以實現交互效果。 Python 是一種高階編程語言，近年來被拿來做許多機器學習和深度學習的開發與模型訓練。
